## Credit:
https://github.com/amrrs/stable-diffusion-prompt-inpainting

Uncomment the following lines whenever setting up the notebook for the first time

In [2]:
# ! git lfs install

In [3]:
# ! git clone https://github.com/timojl/clipseg 

In [4]:
# ! pip install diffusers -q

In [5]:
# ! pip install transformers -q -UU ftfy gradio  

In [5]:
# ! pip install git+https://github.com/openai/CLIP.git -q

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core,wincred).
Your token has been saved to C:\Users\simeo\.huggingface\token
Login successful


In [6]:
%cd clipseg
import torch
import requests
import cv2 as cv
from models.clipseg import CLIPDensePredT
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
%cd ..

C:\Users\simeo\Desktop\CS 543\Final Project\DreamDance\stable-diffusion-prompt-inpainting\clipseg
C:\Users\simeo\Desktop\CS 543\Final Project\DreamDance\stable-diffusion-prompt-inpainting


In [7]:
from io import BytesIO

import os
import random
from torch.cuda.amp import autocast
import requests
import PIL
import torch
from diffusers import StableDiffusionInpaintPipeline as StableDiffusionInpaintPipeline

In [8]:
def make_square(image):
    width, height = image.size
    biggest_side = 0
    
    left_pad = 0
    top_pad = 0
    if (width > height):
        biggest_side = width
        top_pad = int((width - height)/2)
    else:
        biggest_side = height
        left_pad = int((height - width)/2)
    
    result = Image.new(image.mode, (biggest_side, biggest_side), (128, 128, 128))
    result.paste(image, (left_pad, top_pad))
    
    return result

In [9]:
def get_model_and_pipe():
    model = CLIPDensePredT(version='ViT-B/16', reduce_dim=64)
    model.eval();
    
    model.load_state_dict(torch.load('clipseg/weights/rd64-uni.pth', map_location=torch.device('cuda')), strict=False);
    
    device = "cuda"
    pipe = StableDiffusionInpaintPipeline.from_pretrained(
        "runwayml/stable-diffusion-inpainting",
        revision="fp16", 
        torch_dtype=torch.float16,
        use_auth_token=True
    ).to(device)
    
    return model, pipe

In [10]:
def generate_inpainting(image_location, prompt, model, pipe):
    image_name = image_location.split("/")[-1].split(".")[0]
    file_extension = image_location.split(".")[-1]
    filtered_prompt = prompt.translate('<>:\"/\\|?*')
    filtered_prompt = "_".join(prompt.split(" ")).lower()
    
    if (file_extension != "jpg"):
        print("Only jpg images are allowed")
        return None
    preprocessed_dir = "preprocessed"
    mask_dir = "masks"
    output_dir = "output/"+filtered_prompt
    if (not os.path.exists(preprocessed_dir)):
        os.mkdir(preprocessed_dir)
    if (not os.path.exists(mask_dir)):
        os.mkdir(mask_dir)
    if (not os.path.exists(output_dir)):
        os.mkdir(output_dir)
    
    init_image_location = preprocessed_dir + "/"+image_name+"_preprocessed.png"
    mask_image_location = mask_dir +"/"+image_name+"_mask.png"
    output_image_location = output_dir+"/"+image_name+f"_output{random.randint(100000,999999)}.jpg"
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((512, 512)),
    ])
    
    # input_image = cv.imread("model5.png")
    input_image = PIL.Image.open(image_location, mode='r', formats=None)
    input_image = make_square(input_image)

    input_image.convert("RGB").resize((512, 512)).save(init_image_location, "PNG")
    img = transform(input_image).unsqueeze(0)
    
    prompts = ["person"]
    # predict
    with torch.no_grad():
        preds = model(img.repeat(len(prompts),1,1,1), prompts)[0]
        
    plt.imsave(mask_image_location, torch.sigmoid(1 - preds[0][0]))
    
    img2 = cv.imread(mask_image_location)
    gray_image = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
    (thresh, bw_image) = cv.threshold(gray_image, 100, 255, cv.THRESH_BINARY)

    # For debugging only:
    cv.imwrite(mask_image_location,bw_image)

    # fix color format
    cv.cvtColor(bw_image, cv.COLOR_BGR2RGB)

    Image.fromarray(bw_image)
    
    init_image = Image.open(init_image_location)
    mask = Image.open(mask_image_location)
    
    with autocast(True):
        images = pipe(prompt=prompt, image=init_image, mask_image=mask)
    
    images[0][0].save(output_image_location, "JPEG")
    return images[0][0]

In [11]:
model, pipe = get_model_and_pipe()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
prompts = ["beautiful hawaiian beach", "modern city plaza in the morning", "terrifying martian landscape", "crazy nightclub"]
for prompt in prompts:
    for i in range(1,7):
        image = f"samples/model{i}.jpg"
        generate_inpainting(image, prompt, model, pipe)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]